In [1]:
import os
os.chdir('..')
os.getcwd()

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import pickle 
import numpy as np
import os

In [3]:
parent_dir = os.path.abspath('..')
test_file = '/data/cora/cora.pkl'
with open(parent_dir+test_file, 'rb') as f:
    [adj, features, labels] = pickle.load(f)

In [4]:
adj

<2708x2708 sparse matrix of type '<class 'numpy.int64'>'
	with 10556 stored elements in Compressed Sparse Row format>

In [5]:
X = features.todense()

In [6]:
classes = np.argmax(labels, axis=1)

In [7]:
from core.DimReduction import DimReduction

In [8]:
dim_red = DimReduction(n_components=2)

In [9]:
X_emb = dim_red.fit_transform(X, 'tsne')

In [10]:
from util.plot_bokeh_embedding import plot_embedding
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [11]:
target_names = [str(i) for i in range(7)]
named_labels = [target_names[l] for l in classes]

In [12]:
plot_embedding(X_emb, classes, named_labels)

In [13]:
from sklearn.manifold import SpectralEmbedding

In [14]:
embedding = SpectralEmbedding(n_components=2, affinity='precomputed')

In [15]:
X_spectral = embedding.fit_transform(adj.todense())

/Users/signapoop/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:237: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [16]:
plot_embedding(X_spectral, classes, named_labels)

In [64]:
W = adj.toarray()

In [65]:
D = np.sum(W, axis=1)
D.shape

(2708,)

In [66]:
D = np.diag(D)

In [67]:
D.shape

(2708, 2708)

In [68]:
L = D - W

In [76]:
foo = np.trace(np.matmul(np.matmul(X_spectral.T, L), X_spectral))
foo

1.78447981793407e-18

In [77]:
foo = np.trace(np.matmul(np.matmul(X_emb.T, L), X_emb))
foo

496008.9065489512

In [26]:
from scipy.sparse.csgraph import connected_components

In [27]:
n_connected_components, _ = connected_components(W)
n_connected_components

78

In [52]:
import torch
if torch.cuda.is_available():
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    dtypeDouble = torch.cuda.DoubleTensor
else:
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    dtypeDouble = torch.DoubleTensor

In [86]:
def graph_torch_loss(adj, X_emb):
    from scipy.sparse.csgraph import laplacian
    L = laplacian(adj, normed=False, return_diag=False)

    L = torch.from_numpy(L).type(dtypeDouble)
    cut = np.trace(torch.mm(torch.mm(torch.t(X_emb), L), X_emb))
    return cut

In [81]:
X_test = torch.from_numpy(X_emb).type(dtypeDouble)
graph_torch_loss(adj, X_test)

496008.9065489512

In [82]:
from scipy.sparse.csgraph import laplacian
L = laplacian(adj, normed=False, return_diag=False)

In [85]:
L.toarray()

array([[ 3,  0,  0, ...,  0,  0,  0],
       [ 0,  3, -1, ...,  0,  0,  0],
       [ 0, -1,  5, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  1,  0,  0],
       [ 0,  0,  0, ...,  0,  4, -1],
       [ 0,  0,  0, ...,  0, -1,  4]], dtype=int64)